In [ ]:
import os
import chromadb
import utils
import openai
openai.api_key = utils.get_openai_api_key()

In [ ]:
from llama_index import SimpleDirectoryReader

# Load PDF documents
documents = SimpleDirectoryReader(input_files=["./IPCC_AR6_WGII_Chapter03.pdf", "./eBook-How-to-Build-a-Career-in-AI.pdf"]).load_data()


In [ ]:
type(documents)

### Basic rag pipeline

In [ ]:
from llama_index import Document

In [ ]:
dir(Document)

In [ ]:
doc=Document(text="/n/n".join([docs.text for docs in documents]))

In [ ]:
pip freeze | grep "llama"

In [ ]:
import llama_index

In [ ]:
dir(llama_index.vector_stores)

In [1]:
from llama_index.vector_stores import ChromaVectorStore
from llama_index.storage.docstore import SimpleDocumentStore
from llama_index.storage.index_store import SimpleIndexStore
from llama_index import StorageContext
import chromadb

# Setup ChromaDB
chroma_client = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = chroma_client.get_or_create_collection("data")

# Configure storage context
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(
    vector_store=vector_store,
    docstore=SimpleDocumentStore(),
    index_store=SimpleIndexStore()
)



ModuleNotFoundError: No module named 'llama_index.vector_stores'

### Storing and indexing the vectorstore

In [ ]:
ls

In [ ]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader
from llama_index import ServiceContext
from llama_index.llms import OpenAI



docs = SimpleDirectoryReader(input_files=["./IPCC_AR6_WGII_Chapter03.pdf"]).load_data()



llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)
service_context = ServiceContext.from_defaults(
    llm=llm, embed_model="local:BAAI/bge-small-en-v1.5"
)
# index = VectorStoreIndex.from_documents([doc],
#                                         service_context=service_context)
# ✅ Step 2: Create an index using ChromaDB as a vector store
# index = VectorStoreIndex.from_documents(docs, storage_context=storage_context)

In [ ]:
index = VectorStoreIndex.from_documents(
    [doc], storage_context=storage_context, service_context=service_context
)


In [ ]:
print(index.index_id)

In [ ]:
index.storage_context.persist(persist_dir="./chroma_db")


In [ ]:
from llama_index import StorageContext, load_index_from_storage
from llama_index.vector_stores import ChromaVectorStore
import chromadb

# # Load ChromaDB
# chroma_client = chromadb.PersistentClient(path="./chroma_db")
# chroma_collection = chroma_client.get_or_create_collection("data")

# # Load the persisted storage context with the vector store
# vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
# storage_context = StorageContext.from_defaults(vector_store=vector_store, persist_dir="./chroma_db")

# id='e87dbac9-0318-449c-82ad-2e09c65c4ebd'
# # Load index from storage
# index = load_index_from_storage(storage_context,index_id=[id])

# # Create Query Engine
# query_engine = index.as_query_engine()

# # Run query
# query = "What are the key findings from Chapter 3 of the IPCC report?"
# response = query_engine.query(query)

# print(response)


In [ ]:
dir(llama_index)

In [ ]:
from llama_index import load_indices_from_storage

In [ ]:
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store, persist_dir="./chroma_db")

# Use the exact index_id from your persisted index
index_id = "e87dbac9-0318-449c-82ad-2e09c65c4ebd"
indices = load_indices_from_storage(storage_context, index_ids=[index_id])

# Extract the index
index = indices[0]

# Create Query Engine
query_engine = index.as_query_engine()

# Run query
query = "What are the key findings from Chapter 3 of the IPCC report?"
response = query_engine.query(query)


In [ ]:
?load_indices_from_storage

In [ ]:
query_engine = index.as_query_engine()

In [ ]:
response = query_engine.query(
    "What are steps to take when finding projects to build your experience?"
)
print(str(response))